# Banana Bot Baseline Code
## Samuel Horovatin, 11185403, sch923

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, datetime
import math

EPOCHS = 20
BATCH = 8
OPTIMIZER = 'adam'
LOSS = 'mse'

def MRD(y_true, y_pred):
    return tf.keras.backend.sum(tf.keras.backend.abs(y_pred-y_true)/y_true)

def one_FVU(y_true, y_pred):
    y_mean = tf.keras.backend.mean(y_true)
    return 1 - (tf.keras.backend.sum(tf.keras.backend.square(y_true - y_pred)) / tf.keras.backend.sum(tf.keras.backend.square(y_true - y_mean)))

METRICS = [MRD, one_FVU]

In [5]:
# Constants for Data Imports
# DATA_PATH = 'data'
DATA_PATH = 'G:\\shoro\\OneDrive - Personal\\OneDrive\School\\2020-2021\\CMPT 898\\CMPT898_Repo\\Project\\data\\bananas\\'
LABEL_PATH = "Estu.csv"
VALIDATION_SPLIT = 0.3
SEED = 1983
TARGET_SIZE = (320,320)

# Gets labels from label csv
data_label_df = pd.read_csv(LABEL_PATH)

# Create training and testing datasets
dat_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=VALIDATION_SPLIT)

train_datagen = dat_gen.flow_from_dataframe(
    dataframe=data_label_df,
    directory=DATA_PATH,
    x_col='File Name',
    y_col='Weight',
    target_size=TARGET_SIZE,
    batch_size=BATCH,
    class_mode='raw',
    shuffle=True,
    seed=SEED,
    subset='training'
)

valid_datagen = dat_gen.flow_from_dataframe(
    dataframe=data_label_df,
    directory=DATA_PATH,
    x_col='File Name',
    y_col='Weight',
    target_size=TARGET_SIZE,
    batch_size=BATCH,
    class_mode='raw',
    shuffle=True,
    seed=SEED,
    subset="validation"
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_datagen = test_datagen.flow_from_dataframe(
    dataframe=data_label_df,
    directory=DATA_PATH,
    x_col='File Name',
    y_col='Weight',
    target_size=TARGET_SIZE,
    batch_size=BATCH,
    class_mode='raw',
    shuffle=True,
    seed=SEED
)

Found 399 validated image filenames.
Found 170 validated image filenames.
Found 569 validated image filenames.


In [6]:
# Trains the model given the constant parameters and returns the model and the fit history
def train_model(model, train_datagen, test_datagen, valid_datagen):
  model.compile(optimizer=OPTIMIZER,
                loss=LOSS,
                metrics=METRICS)

  csv_logger = tf.keras.callbacks.CSVLogger('training.log', separator=',')
  early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.03, mode='min', patience=8)

  STEP_SIZE_TRAIN=train_datagen.n//train_datagen.batch_size
  STEP_SIZE_VALID=valid_datagen.n//valid_datagen.batch_size
  STEP_SIZE_TEST=test_datagen.n//test_datagen.batch_size

  history = model.fit(x=train_datagen,
                      steps_per_epoch=STEP_SIZE_TRAIN,
                      validation_data=valid_datagen,
                      validation_steps=STEP_SIZE_VALID, 
                      epochs=EPOCHS,
                      callbacks=[csv_logger])
        
  return (model, history)

In [7]:
def regression_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=6, 
        kernel_size=5, 
        activation='relu', 
        input_shape=(320, 320, 3)))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Conv2D(
        filters=16,
        kernel_size=5, 
        activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(120, activation='relu'))
    model.add(tf.keras.layers.Dense(84, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    return model

train_model(regression_model(), train_datagen, test_datagen, valid_datagen)[0]

Epoch 1/20
49/49 [==============================] - 11s 230ms/step - loss: 662.1063 - MRD: 7.8857 - one_FVU: -19.0289 - val_loss: 28.2508 - val_MRD: 5.9136 - val_one_FVU: -6.4160
Epoch 2/20
49/49 [==============================] - 4s 87ms/step - loss: 552.0653 - MRD: 6.8972 - one_FVU: -15.0679 - val_loss: 623.3693 - val_MRD: 32.7808 - val_one_FVU: -207.3793
Epoch 3/20
49/49 [==============================] - 4s 87ms/step - loss: 410.1734 - MRD: 7.7575 - one_FVU: -19.8305 - val_loss: 420.5046 - val_MRD: 24.5527 - val_one_FVU: -138.1605
Epoch 4/20
49/49 [==============================] - 4s 82ms/step - loss: 249.2505 - MRD: 7.2754 - one_FVU: -195.1896 - val_loss: 23.1052 - val_MRD: 5.1429 - val_one_FVU: -5.7787
Epoch 5/20
49/49 [==============================] - 4s 85ms/step - loss: 205.2430 - MRD: 5.7187 - one_FVU: -11.4990 - val_loss: 42.8474 - val_MRD: 6.9267 - val_one_FVU: -12.2486
Epoch 6/20
49/49 [==============================] - 5s 94ms/step - loss: 103.6394 - MRD: 4.3956 - one_F